# 📘 Preprocessing Notebook – Tutas Recommender

This notebook handles the preprocessing pipeline **after feature engineering in Google Cloud BigQuery**.  
The feature-engineered dataset (`tutas_recommender_training_features.csv`) is exported from BigQuery and used here as input.

---

## 🎯 Objective
- Load the feature-engineered dataset from BigQuery export.
- Clean and prepare the data for machine learning.
- Split into training and testing sets (`X_train`, `X_test`, `y_train`, `y_test`).
- Save processed files under `dataset/processed_2/` for model training.

---

📌 **Note**:  
- The dataset is **synthetic** and was created for demonstration purposes.  
- No real user data is involved.  


## 1. Import Dependencies

We start by importing the required libraries for preprocessing:

- **pandas** → to handle and manipulate tabular datasets.  
- **sklearn.preprocessing.StandardScaler** → to standardize numerical features (zero mean, unit variance).  
- **sklearn.model_selection.train_test_split** → to split the dataset into training and testing subsets.

These libraries will be used throughout the notebook to clean, transform, and prepare the data for model training.


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## 2. Load the Feature-Engineered Dataset

We load the dataset exported from **BigQuery** (`tutas_recommender_training_features.csv`).  
This dataset contains feature-engineered records representing student–tutor interactions.

After loading:
- Display the first few rows with `head()` to get an overview.  
- Use `info()` to check data types and non-null counts.  
- Use `isnull().sum()` to identify any missing values in each column.


In [ ]:
# Load the feature-engineered dataset exported from BigQuery
df = pd.read_csv('../data/dataset/processed_1/tutas_recommender_training_features.csv')

# Preview the first 5 rows to understand dataset structure
print(df.head())

# Display data types, column info, and non-null counts
df.info()

# Check for missing values in each column
df.isnull().sum()


## 3. Identify Numerical Features

We select all columns with numeric data types (`int64`, `float64`) for further preprocessing.  
This step is important because numerical features will later be **standardized** using `StandardScaler` to improve model performance.

After selecting:
- Print the list of numerical features.  
- Use `describe()` to view basic statistics (mean, std, min, max, quartiles) for these features.


In [ ]:
# Select columns with numeric data types (int64, float64)
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Print the list of numerical feature names
print("Fitur Numerikal : ", list(numerical_cols))

# Show descriptive statistics for numerical features
print(df[numerical_cols].describe())


## 4. Encode Boolean Features

Some columns contain boolean values (`True` / `False`).  
Since most machine learning algorithms expect numeric inputs, we convert these boolean features into integers:
- `True` → 1  
- `False` → 0  

This ensures all features are numeric and ready for preprocessing or scaling.


In [ ]:
# Convert all boolean columns into integer values (True=1, False=0)
for col in df.select_dtypes(include=['bool']).columns:
    df[col] = df[col].astype(int)


## 5. Feature Scaling

We apply **StandardScaler** to normalize numerical features.  
- StandardScaler standardizes features by removing the mean and scaling to unit variance.  
- It works well because the data distribution is approximately normal and does not contain many outliers.  
- If outliers were present, a **RobustScaler** would be more appropriate.

Steps:
1. Separate features (`X`) and target (`y`).  
2. Select numerical columns to be scaled.  
3. Apply `StandardScaler` only to the selected numerical features.


In [ ]:
# Separate features (X) and target label (y)
X = df.drop(columns=['label', 'id_murid', 'id_tutor'])  # remove label and ID columns
y = df['label']

# Define numerical columns to be standardized
numerical_cols = ['feedback_score', 'average_rating_tutor', 'total_jam_ngajar_tutor']

# Initialize StandardScaler
scaler = StandardScaler()

# Apply scaling to the selected numerical features
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])


## 6. Train–Test Split

We now split the dataset into **training** and **testing** subsets:

- **Training set (80%)** → used to train the machine learning model.  
- **Testing set (20%)** → held out for evaluation, to check generalization performance.  

After splitting, we save each subset as CSV files under `dataset/processed_2/`:
- `X_train.csv`, `X_test.csv`
- `y_train.csv`, `y_test.csv`


In [ ]:
# Split dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Save the split datasets into processed 2 folder
X_train.to_csv('../data/dataset/processed 2/X_train.csv', index=False)
X_test.to_csv('../data/dataset/processed 2/X_test.csv', index=False)
y_train.to_csv('../data/dataset/processed 2/y_train.csv', index=False)
y_test.to_csv('../data/dataset/processed 2/y_test.csv', index=False)
